In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.svm import SVC

# import data
X_train = pd.read_csv('DT/stack_train4.csv', delimiter= ",",header=0)
X = X_train.as_matrix()
#X = X[:,0:3]
print(X.shape)
X_test = pd.read_csv('DT/stack_test4.csv', delimiter= ",",header=0)
X_test = X_test.as_matrix()
#X_test =  X_test[:,0:3]
print(X_test.shape)
y_train = pd.read_csv('DT/y_train4.csv', delimiter= ",",header=0)
y = y_train.as_matrix()
print(y.shape)
y = y.reshape(X_train.shape[0])
# print(y)
y_test = pd.read_csv('DT/y_test4.csv', delimiter= ",",header=0)
y_test = y_test.as_matrix()
print(y_test.shape)
# scaler = StandardScaler()
# X = scaler.fit_transform(X)


ModuleNotFoundError: No module named 'xgboost'

In [31]:
#function for grid search
class CustomGridCV(object):
    def __init__(self, X, y, model, metric, griddata, cv=5):
        self.X = X
        self.y = y
        self.model = model
        self.metric = metric
        self.params = self.gridpoints(griddata)
        self.cv = cv
        self.bestScore = None
        self.bestParams = None
        
    def gridpoints(self, data):
        newparams = [{}]
        for k in data.keys():
            params = newparams
            newparams = []
            for v in data[k]:
                for param in params:
                    item = param.copy()
                    item[k]=v
                    newparams.append(item)           
        return newparams
    
    def GridSearch(self):
        for param in self.params:
            self.model.set_params(**param)
            score = self.KFoldScore()
            if self.bestScore==None or self.bestScore<score:
                self.bestScore = score
                self.bestParams = param
            print("Score: {0:.5f}, Params: {1}".format(score,param))
    
    def KFoldScore(self):
        #kf = KFold(n_splits=5, shuffle=True, random_state=2)
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
        y_pred = np.zeros(len(self.y))

        for train_index, test_index in kf.split(self.X,self.y):
            train_X, test_X = self.X[train_index], self.X[test_index]
#             print(train_X)
            train_y, test_y = self.y[train_index], self.y[test_index]
            self.model.fit(train_X,train_y)
            #y_pred[test_index] = self.model.predict_proba(test_X)[:,1]
            y_pred[test_index] = self.model.predict(test_X)
            #print y_pred[test_index]

        return self.metric(self.y,y_pred)
    
    def Best(self):
        return self.bestScore, self.bestParams

In [37]:
#XG boost
model = XGBClassifier()
griddata = {'max_depth':range(3,8),
            'min_child_weight':range(1,5),
            'learning_rate': np.arange(0.01,0.2,0.02)
            }
GCV = CustomGridCV(X, y, model, accuracy_score, griddata)
GCV.GridSearch()
print("Best Params:")
print(GCV.Best())

Score: 0.74901, Params: {'max_depth': 3, 'min_child_weight': 1, 'learning_rate': 0.01}
Score: 0.74758, Params: {'max_depth': 4, 'min_child_weight': 1, 'learning_rate': 0.01}
Score: 0.74653, Params: {'max_depth': 5, 'min_child_weight': 1, 'learning_rate': 0.01}
Score: 0.74578, Params: {'max_depth': 6, 'min_child_weight': 1, 'learning_rate': 0.01}
Score: 0.74503, Params: {'max_depth': 7, 'min_child_weight': 1, 'learning_rate': 0.01}
Score: 0.74901, Params: {'max_depth': 3, 'min_child_weight': 2, 'learning_rate': 0.01}
Score: 0.74781, Params: {'max_depth': 4, 'min_child_weight': 2, 'learning_rate': 0.01}
Score: 0.74631, Params: {'max_depth': 5, 'min_child_weight': 2, 'learning_rate': 0.01}
Score: 0.74578, Params: {'max_depth': 6, 'min_child_weight': 2, 'learning_rate': 0.01}
Score: 0.74541, Params: {'max_depth': 7, 'min_child_weight': 2, 'learning_rate': 0.01}
Score: 0.74893, Params: {'max_depth': 3, 'min_child_weight': 3, 'learning_rate': 0.01}
Score: 0.74796, Params: {'max_depth': 4, 'm

Score: 0.74511, Params: {'max_depth': 6, 'min_child_weight': 1, 'learning_rate': 0.089999999999999983}
Score: 0.74346, Params: {'max_depth': 7, 'min_child_weight': 1, 'learning_rate': 0.089999999999999983}
Score: 0.74863, Params: {'max_depth': 3, 'min_child_weight': 2, 'learning_rate': 0.089999999999999983}
Score: 0.74721, Params: {'max_depth': 4, 'min_child_weight': 2, 'learning_rate': 0.089999999999999983}
Score: 0.74653, Params: {'max_depth': 5, 'min_child_weight': 2, 'learning_rate': 0.089999999999999983}
Score: 0.74511, Params: {'max_depth': 6, 'min_child_weight': 2, 'learning_rate': 0.089999999999999983}
Score: 0.74376, Params: {'max_depth': 7, 'min_child_weight': 2, 'learning_rate': 0.089999999999999983}
Score: 0.74863, Params: {'max_depth': 3, 'min_child_weight': 3, 'learning_rate': 0.089999999999999983}
Score: 0.74758, Params: {'max_depth': 4, 'min_child_weight': 3, 'learning_rate': 0.089999999999999983}
Score: 0.74638, Params: {'max_depth': 5, 'min_child_weight': 3, 'learning

Score: 0.74316, Params: {'max_depth': 7, 'min_child_weight': 1, 'learning_rate': 0.16999999999999998}
Score: 0.74818, Params: {'max_depth': 3, 'min_child_weight': 2, 'learning_rate': 0.16999999999999998}
Score: 0.74758, Params: {'max_depth': 4, 'min_child_weight': 2, 'learning_rate': 0.16999999999999998}
Score: 0.74623, Params: {'max_depth': 5, 'min_child_weight': 2, 'learning_rate': 0.16999999999999998}
Score: 0.74488, Params: {'max_depth': 6, 'min_child_weight': 2, 'learning_rate': 0.16999999999999998}
Score: 0.74354, Params: {'max_depth': 7, 'min_child_weight': 2, 'learning_rate': 0.16999999999999998}
Score: 0.74826, Params: {'max_depth': 3, 'min_child_weight': 3, 'learning_rate': 0.16999999999999998}
Score: 0.74766, Params: {'max_depth': 4, 'min_child_weight': 3, 'learning_rate': 0.16999999999999998}
Score: 0.74593, Params: {'max_depth': 5, 'min_child_weight': 3, 'learning_rate': 0.16999999999999998}
Score: 0.74436, Params: {'max_depth': 6, 'min_child_weight': 3, 'learning_rate': 0

In [62]:
#XG boost
model = XGBClassifier(max_depth=3, min_child_weight=1, learning_rate=0.03, n_estimators=1000)
trained_model = model.fit(X, y)
predictions = trained_model.predict(X_test)
print('Test accuracy is \n',accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))


Test accuracy is 
 0.846669861045
             precision    recall  f1-score   support

        1.0       0.92      0.93      0.92      3701
        2.0       0.75      0.81      0.78      1768
        3.0       0.71      0.53      0.61       526
        4.0       0.68      0.57      0.62       173
        5.0       0.75      0.70      0.72        93

avg / total       0.84      0.85      0.84      6261



In [47]:
#DT
model = DecisionTreeClassifier()
griddata = {"min_samples_split":range(2,50),
            "max_features": range(1,4),
#            "max_depth": range(1,10)
            }

# Grid Search for the best parameters
GCV = CustomGridCV(X, y, model, accuracy_score, griddata)
GCV.GridSearch()

print("Best Params:")
print(GCV.Best())

Score: 0.74226, Params: {'min_samples_split': 2, 'max_features': 1}
Score: 0.74226, Params: {'min_samples_split': 3, 'max_features': 1}
Score: 0.74226, Params: {'min_samples_split': 4, 'max_features': 1}
Score: 0.74219, Params: {'min_samples_split': 5, 'max_features': 1}
Score: 0.74189, Params: {'min_samples_split': 6, 'max_features': 1}
Score: 0.74211, Params: {'min_samples_split': 7, 'max_features': 1}
Score: 0.74211, Params: {'min_samples_split': 8, 'max_features': 1}
Score: 0.74219, Params: {'min_samples_split': 9, 'max_features': 1}
Score: 0.74226, Params: {'min_samples_split': 10, 'max_features': 1}
Score: 0.74189, Params: {'min_samples_split': 11, 'max_features': 1}
Score: 0.74181, Params: {'min_samples_split': 12, 'max_features': 1}
Score: 0.74181, Params: {'min_samples_split': 13, 'max_features': 1}
Score: 0.74166, Params: {'min_samples_split': 14, 'max_features': 1}
Score: 0.74219, Params: {'min_samples_split': 15, 'max_features': 1}
Score: 0.74151, Params: {'min_samples_spli

Score: 0.74414, Params: {'min_samples_split': 29, 'max_features': 3}
Score: 0.74451, Params: {'min_samples_split': 30, 'max_features': 3}
Score: 0.74399, Params: {'min_samples_split': 31, 'max_features': 3}
Score: 0.74406, Params: {'min_samples_split': 32, 'max_features': 3}
Score: 0.74481, Params: {'min_samples_split': 33, 'max_features': 3}
Score: 0.74376, Params: {'min_samples_split': 34, 'max_features': 3}
Score: 0.74503, Params: {'min_samples_split': 35, 'max_features': 3}
Score: 0.74429, Params: {'min_samples_split': 36, 'max_features': 3}
Score: 0.74548, Params: {'min_samples_split': 37, 'max_features': 3}
Score: 0.74496, Params: {'min_samples_split': 38, 'max_features': 3}
Score: 0.74556, Params: {'min_samples_split': 39, 'max_features': 3}
Score: 0.74541, Params: {'min_samples_split': 40, 'max_features': 3}
Score: 0.74444, Params: {'min_samples_split': 41, 'max_features': 3}
Score: 0.74556, Params: {'min_samples_split': 42, 'max_features': 3}
Score: 0.74481, Params: {'min_samp

In [63]:
#DT
model = DecisionTreeClassifier(min_samples_split=48, max_features=3)
trained_model = model.fit(X, y)
predictions = trained_model.predict(X_test)
print('Test accuracy is \n',accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

Test accuracy is 
 0.849864238939
             precision    recall  f1-score   support

        1.0       0.92      0.93      0.92      3701
        2.0       0.76      0.80      0.78      1768
        3.0       0.72      0.57      0.64       526
        4.0       0.69      0.57      0.63       173
        5.0       0.77      0.70      0.73        93

avg / total       0.85      0.85      0.85      6261



In [49]:
#RF
model = RandomForestClassifier()
griddata = {"n_estimators":range(100,150),
            "max_features": range(1,4),
#             "max_depth": range(1,10)
            }

# Grid Search for the best parameters
GCV = CustomGridCV(X, y, model, accuracy_score, griddata)
GCV.GridSearch()

print("Best Params:")
print(GCV.Best())

Score: 0.74279, Params: {'n_estimators': 100, 'max_features': 1}
Score: 0.74399, Params: {'n_estimators': 101, 'max_features': 1}
Score: 0.74264, Params: {'n_estimators': 102, 'max_features': 1}
Score: 0.74309, Params: {'n_estimators': 103, 'max_features': 1}
Score: 0.74256, Params: {'n_estimators': 104, 'max_features': 1}
Score: 0.74211, Params: {'n_estimators': 105, 'max_features': 1}
Score: 0.74294, Params: {'n_estimators': 106, 'max_features': 1}
Score: 0.74309, Params: {'n_estimators': 107, 'max_features': 1}
Score: 0.74256, Params: {'n_estimators': 108, 'max_features': 1}
Score: 0.74249, Params: {'n_estimators': 109, 'max_features': 1}
Score: 0.74219, Params: {'n_estimators': 110, 'max_features': 1}
Score: 0.74279, Params: {'n_estimators': 111, 'max_features': 1}
Score: 0.74309, Params: {'n_estimators': 112, 'max_features': 1}
Score: 0.74384, Params: {'n_estimators': 113, 'max_features': 1}
Score: 0.74264, Params: {'n_estimators': 114, 'max_features': 1}
Score: 0.74264, Params: {

Score: 0.74339, Params: {'n_estimators': 127, 'max_features': 3}
Score: 0.74241, Params: {'n_estimators': 128, 'max_features': 3}
Score: 0.74189, Params: {'n_estimators': 129, 'max_features': 3}
Score: 0.74309, Params: {'n_estimators': 130, 'max_features': 3}
Score: 0.74279, Params: {'n_estimators': 131, 'max_features': 3}
Score: 0.74256, Params: {'n_estimators': 132, 'max_features': 3}
Score: 0.74271, Params: {'n_estimators': 133, 'max_features': 3}
Score: 0.74301, Params: {'n_estimators': 134, 'max_features': 3}
Score: 0.74286, Params: {'n_estimators': 135, 'max_features': 3}
Score: 0.74294, Params: {'n_estimators': 136, 'max_features': 3}
Score: 0.74361, Params: {'n_estimators': 137, 'max_features': 3}
Score: 0.74301, Params: {'n_estimators': 138, 'max_features': 3}
Score: 0.74286, Params: {'n_estimators': 139, 'max_features': 3}
Score: 0.74301, Params: {'n_estimators': 140, 'max_features': 3}
Score: 0.74339, Params: {'n_estimators': 141, 'max_features': 3}
Score: 0.74241, Params: {

In [64]:
#RF
model = RandomForestClassifier(n_estimators=200, max_features=4)
trained_model = model.fit(X, y)
predictions = trained_model.predict(X_test)
print('Test accuracy is \n',accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

Test accuracy is 
 0.848586487782
             precision    recall  f1-score   support

        1.0       0.92      0.93      0.92      3701
        2.0       0.76      0.80      0.78      1768
        3.0       0.71      0.56      0.63       526
        4.0       0.70      0.57      0.62       173
        5.0       0.75      0.67      0.70        93

avg / total       0.85      0.85      0.85      6261



In [61]:
#LR
model = LogisticRegression()
griddata = {"C": np.arange(0.01,10,0.01)
            }

# Grid Search for the best parameters
GCV = CustomGridCV(X, y, model, accuracy_score, griddata)
GCV.GridSearch()

print("Best Params:")
print(GCV.Best())

Score: 0.63314, Params: {'C': 0.01}
Score: 0.63404, Params: {'C': 0.02}
Score: 0.63584, Params: {'C': 0.029999999999999999}
Score: 0.63876, Params: {'C': 0.040000000000000001}
Score: 0.64109, Params: {'C': 0.050000000000000003}
Score: 0.64146, Params: {'C': 0.060000000000000005}
Score: 0.64356, Params: {'C': 0.069999999999999993}
Score: 0.64378, Params: {'C': 0.080000000000000002}
Score: 0.64806, Params: {'C': 0.089999999999999997}
Score: 0.64955, Params: {'C': 0.099999999999999992}
Score: 0.64955, Params: {'C': 0.11}
Score: 0.64955, Params: {'C': 0.12}
Score: 0.64970, Params: {'C': 0.13}
Score: 0.64963, Params: {'C': 0.14000000000000001}
Score: 0.64955, Params: {'C': 0.15000000000000002}
Score: 0.64955, Params: {'C': 0.16}
Score: 0.64955, Params: {'C': 0.17000000000000001}
Score: 0.64955, Params: {'C': 0.18000000000000002}
Score: 0.64955, Params: {'C': 0.19}
Score: 0.64970, Params: {'C': 0.20000000000000001}
Score: 0.64985, Params: {'C': 0.21000000000000002}
Score: 0.65000, Params: {'

Score: 0.65008, Params: {'C': 1.79}
Score: 0.65008, Params: {'C': 1.8}
Score: 0.65008, Params: {'C': 1.8100000000000001}
Score: 0.65008, Params: {'C': 1.8200000000000001}
Score: 0.65008, Params: {'C': 1.8300000000000001}
Score: 0.65008, Params: {'C': 1.8400000000000001}
Score: 0.65008, Params: {'C': 1.8500000000000001}
Score: 0.65008, Params: {'C': 1.8600000000000001}
Score: 0.65008, Params: {'C': 1.8700000000000001}
Score: 0.65008, Params: {'C': 1.8800000000000001}
Score: 0.65008, Params: {'C': 1.8900000000000001}
Score: 0.65008, Params: {'C': 1.9000000000000001}
Score: 0.65008, Params: {'C': 1.9100000000000001}
Score: 0.65008, Params: {'C': 1.9200000000000002}
Score: 0.65008, Params: {'C': 1.9299999999999999}
Score: 0.65008, Params: {'C': 1.9399999999999999}
Score: 0.65008, Params: {'C': 1.95}
Score: 0.65008, Params: {'C': 1.96}
Score: 0.65008, Params: {'C': 1.97}
Score: 0.65008, Params: {'C': 1.98}
Score: 0.65008, Params: {'C': 1.99}
Score: 0.65008, Params: {'C': 2.0}
Score: 0.65008

Score: 0.65120, Params: {'C': 3.54}
Score: 0.65120, Params: {'C': 3.5499999999999998}
Score: 0.65120, Params: {'C': 3.5600000000000001}
Score: 0.65120, Params: {'C': 3.5699999999999998}
Score: 0.65120, Params: {'C': 3.5800000000000001}
Score: 0.65120, Params: {'C': 3.5899999999999999}
Score: 0.65120, Params: {'C': 3.5999999999999996}
Score: 0.65120, Params: {'C': 3.6099999999999999}
Score: 0.65120, Params: {'C': 3.6199999999999997}
Score: 0.65158, Params: {'C': 3.6299999999999999}
Score: 0.65158, Params: {'C': 3.6399999999999997}
Score: 0.65158, Params: {'C': 3.6499999999999999}
Score: 0.65158, Params: {'C': 3.6599999999999997}
Score: 0.65158, Params: {'C': 3.6699999999999999}
Score: 0.65158, Params: {'C': 3.6799999999999997}
Score: 0.65158, Params: {'C': 3.6899999999999999}
Score: 0.65158, Params: {'C': 3.6999999999999997}
Score: 0.65158, Params: {'C': 3.71}
Score: 0.65158, Params: {'C': 3.7199999999999998}
Score: 0.65158, Params: {'C': 3.73}
Score: 0.65158, Params: {'C': 3.7399999999

Score: 0.65158, Params: {'C': 5.25}
Score: 0.65158, Params: {'C': 5.2599999999999998}
Score: 0.65158, Params: {'C': 5.2699999999999996}
Score: 0.65158, Params: {'C': 5.2800000000000002}
Score: 0.65158, Params: {'C': 5.29}
Score: 0.65158, Params: {'C': 5.2999999999999998}
Score: 0.65158, Params: {'C': 5.3099999999999996}
Score: 0.65158, Params: {'C': 5.3200000000000003}
Score: 0.65158, Params: {'C': 5.3300000000000001}
Score: 0.65158, Params: {'C': 5.3399999999999999}
Score: 0.65158, Params: {'C': 5.3499999999999996}
Score: 0.65158, Params: {'C': 5.3600000000000003}
Score: 0.65158, Params: {'C': 5.3700000000000001}
Score: 0.65158, Params: {'C': 5.3799999999999999}
Score: 0.65158, Params: {'C': 5.3899999999999997}
Score: 0.65158, Params: {'C': 5.3999999999999995}
Score: 0.65158, Params: {'C': 5.4100000000000001}
Score: 0.65158, Params: {'C': 5.4199999999999999}
Score: 0.65158, Params: {'C': 5.4299999999999997}
Score: 0.65158, Params: {'C': 5.4399999999999995}
Score: 0.65158, Params: {'C'

Score: 0.65150, Params: {'C': 6.9500000000000002}
Score: 0.65150, Params: {'C': 6.96}
Score: 0.65150, Params: {'C': 6.9699999999999998}
Score: 0.65150, Params: {'C': 6.9799999999999995}
Score: 0.65150, Params: {'C': 6.9900000000000002}
Score: 0.65150, Params: {'C': 7.0}
Score: 0.65150, Params: {'C': 7.0099999999999998}
Score: 0.65150, Params: {'C': 7.0199999999999996}
Score: 0.65150, Params: {'C': 7.0300000000000002}
Score: 0.65150, Params: {'C': 7.04}
Score: 0.65150, Params: {'C': 7.0499999999999998}
Score: 0.65150, Params: {'C': 7.0599999999999996}
Score: 0.65150, Params: {'C': 7.0700000000000003}
Score: 0.65150, Params: {'C': 7.0800000000000001}
Score: 0.65150, Params: {'C': 7.0899999999999999}
Score: 0.65150, Params: {'C': 7.0999999999999996}
Score: 0.65150, Params: {'C': 7.1100000000000003}
Score: 0.65150, Params: {'C': 7.1200000000000001}
Score: 0.65150, Params: {'C': 7.1299999999999999}
Score: 0.65150, Params: {'C': 7.1399999999999997}
Score: 0.65150, Params: {'C': 7.15000000000

Score: 0.65150, Params: {'C': 8.6400000000000006}
Score: 0.65150, Params: {'C': 8.6500000000000004}
Score: 0.65150, Params: {'C': 8.6600000000000001}
Score: 0.65150, Params: {'C': 8.6699999999999999}
Score: 0.65150, Params: {'C': 8.6799999999999997}
Score: 0.65150, Params: {'C': 8.6899999999999995}
Score: 0.65150, Params: {'C': 8.6999999999999993}
Score: 0.65150, Params: {'C': 8.7100000000000009}
Score: 0.65150, Params: {'C': 8.7200000000000006}
Score: 0.65150, Params: {'C': 8.7300000000000004}
Score: 0.65150, Params: {'C': 8.7400000000000002}
Score: 0.65150, Params: {'C': 8.75}
Score: 0.65150, Params: {'C': 8.7599999999999998}
Score: 0.65150, Params: {'C': 8.7699999999999996}
Score: 0.65150, Params: {'C': 8.7799999999999994}
Score: 0.65150, Params: {'C': 8.7899999999999991}
Score: 0.65150, Params: {'C': 8.8000000000000007}
Score: 0.65150, Params: {'C': 8.8100000000000005}
Score: 0.65150, Params: {'C': 8.8200000000000003}
Score: 0.65150, Params: {'C': 8.8300000000000001}
Score: 0.65150

In [62]:
model = model = LogisticRegression(C=3.63)
trained_model = model.fit(X, y)
predictions = trained_model.predict(X_test)
print('Test accuracy is \n',accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

Test accuracy is 
 0.662878125546
             precision    recall  f1-score   support

        1.0       0.84      0.92      0.88      2306
        2.0       0.50      0.76      0.60      1516
        3.0       0.30      0.03      0.05       892
        4.0       0.00      0.00      0.00       449
        5.0       0.60      0.87      0.71       556

avg / total       0.58      0.66      0.59      5719



/Applications/Anaconda/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
#LR
model = SVC(kernel='rbf')
griddata = {"C": np.logspace(-5,10,10),
            "gamma": np.logspace(-5, 10, 10)
            
            }

# Grid Search for the best parameters
GCV = CustomGridCV(X, y, model, accuracy_score, griddata)
GCV.GridSearch()

print("Best Params:")
print(GCV.Best())

Score: 0.40823, Params: {'C': 1.0000000000000001e-05, 'gamma': 1.0000000000000001e-05}
Score: 0.40823, Params: {'C': 0.00046415888336127822, 'gamma': 1.0000000000000001e-05}
Score: 0.40823, Params: {'C': 0.021544346900318846, 'gamma': 1.0000000000000001e-05}
Score: 0.40823, Params: {'C': 1.0, 'gamma': 1.0000000000000001e-05}
Score: 0.70711, Params: {'C': 46.415888336127821, 'gamma': 1.0000000000000001e-05}
Score: 0.74309, Params: {'C': 2154.4346900318865, 'gamma': 1.0000000000000001e-05}
Score: 0.74339, Params: {'C': 100000.0, 'gamma': 1.0000000000000001e-05}
Score: 0.53474, Params: {'C': 4641588.8336127913, 'gamma': 1.0000000000000001e-05}
Score: 0.59342, Params: {'C': 215443469.00318867, 'gamma': 1.0000000000000001e-05}


In [65]:
#SVM
model = SVC(kernel='rbf',C=1e5, gamma=1e-5)
trained_model = model.fit(X, y)
predictions = trained_model.predict(X_test)
print('Test accuracy is \n',accuracy_score(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

Test accuracy is 
 0.846030985466
             precision    recall  f1-score   support

        1.0       0.92      0.92      0.92      3701
        2.0       0.75      0.81      0.78      1768
        3.0       0.68      0.57      0.62       526
        4.0       0.70      0.51      0.59       173
        5.0       0.81      0.67      0.73        93

avg / total       0.84      0.85      0.84      6261

